# Анализ тональности: Создание датасета и обучение модели

В этом коллабе происходит:
1. Сбор комментариев из VK.com для создания датасета.
2. Обучать модель на основе BERT для классификации тональности (позитив, негатив, нейтрально (дополнительыно: сарказм ирония, смешанные).


## Требования для запуска
1. Доступ к интернету.
2. Токен VK API (мой токен :
219d7b3e219d7b3e219d7b3ed822b0844c2219d219d7b3e467bf727aa63bd40cfb3a0cb).
 **vk** Группа Вк откуда брал комментарии для датасета
4. Для ускорения обучения использовать GPU



В этой ячейке я устанавливаю все необходимые библиотеки, которые понадобятся для работы: vk_api для работы с ВКонтакте, pandas для работы с данными, transformers и torch для обучения модели, а также scikit-learn для различных утилит.

In [1]:
!pip install vk_api pandas transformers torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
import vk_api
import pandas as pd
import re
import time
from datetime import datetime
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pickle
import gc

# Очистка памяти GPU
torch.cuda.empty_cache()
gc.collect()

90

In [3]:

# Списки слов и эмодзи
POSITIVE_WORDS = [
    "отлично", "классно", "супер", "круто", "здорово", "прекрасно", "хорошо", "замечательно", "восхитительно",
    "клево", "похвально", "великолепно", "чудесно", "рад", "счастлив", "люблю", "обожаю", "прикольно", "весело",
    "спасибо", "благодарю", "лучший", "лучше", "приятно", "красиво", "мило", "удивительно", "восторг", "браво",
    "шикарно", "превосходно", "гениально", "потрясающе", "волшебно", "невероятно", "изумительно", "порадовал",
    "вдохновляет", "успех", "победа", "радостно", "смех", "улыбка", "тепло", "ярко", "добро", "класс",
    "восхитительно", "чудо", "сказка", "мечта", "наслаждение", "лапочка", "солнышко", "милота", "прелесть",
    "красота", "чудесный", "обалденно", "праздник", "ликование", "гармония", "энергия", "блеск", "вдохновение",
    "кайф", "наслаждаюсь", "впечатляет", "умиляет", "трогательно", "сердечно", "искры", "фантастика", "здоровский",
    "душевно", "привлекательно", "обаятельно", "эстетично", "элегантно", "роскошно", "грациозно", "пышно",
    "утончённо", "изысканно", "божественно", "ангельски", "рай", "нежность", "сладко", "умиротворение",
    "спокойствие", "идиллия", "позитив", "оптимизм", "жизнерадостно", "лучик", "сияние",
    "молодец", "молодчина", "красавчик", "легенда", "поздравляю", "познавательно", "полезно", "удобно", "талант",
    "профессионал", "гениальность", "восторженно", "восхищаюсь", "гордость", "надежда", "умиление", "трогательно",
    "миленько", "чудесненько", "приятно удивлён", "бесподобно", "великолепный", "хвалю", "брависсимо", "апRewritten cells"
]

# Списки слов и эмодзи
POSITIVE_WORDS = [
    "отлично", "классно", "супер", "круто", "здорово", "прекрасно", "хорошо", "замечательно", "восхитительно",
    "клево", "похвально", "великолепно", "чудесно", "рад", "счастлив", "люблю", "обожаю", "прикольно", "весело",
    "спасибо", "благодарю", "лучший", "лучше", "приятно", "красиво", "мило", "удивительно", "восторг", "браво",
    "шикарно", "превосходно", "гениально", "потрясающе", "волшебно", "невероятно", "изумительно", "порадовал",
    "вдохновляет", "успех", "победа", "радостно", "смех", "улыбка", "тепло", "ярко", "добро", "класс",
    "восхитительно", "чудо", "сказка", "мечта", "наслаждение", "лапочка", "солнышко", "милота", "прелесть",
    "красота", "чудесный", "обалденно", "праздник", "ликование", "гармония", "энергия", "блеск", "вдохновение",
    "кайф", "наслаждаюсь", "впечатляет", "умиляет", "трогательно", "сердечно", "искры", "фантастика", "здоровский",
    "душевно", "привлекательно", "обаятельно", "эстетично", "элегантно", "роскошно", "грациозно", "пышно",
    "утончённо", "изысканно", "божественно", "ангельски", "рай", "нежность", "сладко", "умиротворение",
    "спокойствие", "идиллия", "позитив", "оптимизм", "жизнерадостно", "лучик", "сияние",
    "молодец", "молодчина", "красавчик", "легенда", "поздравляю", "познавательно", "полезно", "удобно", "талант",
    "профессионал", "гениальность", "восторженно", "восхищаюсь", "гордость", "надежда", "умиление", "трогательно",
    "миленько", "чудесненько", "приятно удивлён", "бесподобно", "великолепный", "хвалю", "брависсимо", "аплодисменты",
    "захватывающе", "вдохновляюще", "похвала", "достойно", "впечатляюще", "безупречно", "колоссально", "феноменально",
    "очаровательно", "великолепие", "сногсшибательно", "восхищение", "радуюсь", "ликую", "веселюсь", "доволен",
    "удовольствие", "наслаждаюсь", "умиляюсь", "радует", "ободряет", "поддерживаю", "суперски", "крутецки",
    "восхитительный", "чудненько", "милашка", "солнечно", "тепло", "уютно", "гармонично", "прелестно"
]

NEGATIVE_WORDS = [
    "ужасно", "плохо", "отстой", "грустно", "жаль", "позор", "кошмар", "провал", "разочарование",
    "фу", "беда", "проблема", "хуже", "худший", "зря", "неприятно", "противно", "мерзко", "жутко", "стыдно",
    "гнев", "злюсь", "бесит", "раздражает", "ненавижу", "скучно", "глупо", "обидно", "фигня",
    "дрянь", "муть", "ерунда", "бред", "чушь", "идиотизм", "дурацко", "невыносимо", "отвратительно",
    "гнило", "треш", "страшно", "кринж", "позорище", "дебилизм", "безнадёга", "тоска", "грусть",
    "тупой", "тупое", "тупая", "тупые", "тупость", "тупица", "бездарно", "жалко", "убого", "хреново",
    "погано", "паршиво", "пакостно", "гнусно", "тошно", "против", "мерзость", "гадость", "негодование",
    "озлобление", "ярость", "злоба", "фурия", "отвращение", "мрак", "темнота", "холодно", "безысходность",
    "уныние", "депрессия", "апатия", "лень", "скверно", "плачевно", "трагедия", "катастрофа", "фиаско",
    "облом", "позорный", "стыдоба", "срам", "неудача", "поражение", "крах", "упадок", "отчаяние", "горе",
    "слёзы", "боль", "страдание", "мука", "ад", "пекло", "траур", "паника", "шок", "испуг", "страх",
    "тревога", "нервы", "истерика", "псих", "безумие", "кошмарный", "жуть", "ужасный", "отстойный",
    "тухлятина", "воняет", "смерть", "деградация", "убили", "разрушили", "портят", "издевательство", "таразм",
    "похабщина", "похабно", "безобразие", "уродство", "неудобно", "криво", "косо", "убожество", "дискомфорт",
    "хлам", "мусор", "ненужный", "лишний", "тормозит", "лагает", "глючит", "провально", "шляпа", "дичь",
    "баг", "косяк", "убийство", "выживают", "запрещают", "ограничивают", "ломают", "усложняют", "дикий ужас",
    "нечитаемо", "непонятно", "неуместно", "неадекватно", "непрофессионально", "бездарность", "вредительство",
    "разрушение", "регресс", "упадок", "абсурд", "нелепость", "извращение", "запрет", "невыносимо", "тошнит",
    "противно", "мерзко", "отвратно", "фуу", "фууу", "фуууу", "гадко", "позорно", "стыд", "стыдоба", "срамно",
    "ужасный", "кошмарный", "отвратительный", "мерзкий", "противный", "скверный", "паршивый",
    "тупизм", "идиотский", "кретинизм", "дебильный", "тупиковый", "безысходный", "мракобесие", "хаос",
    "бардак", "беспорядок", "неразбериха", "кавардак", "безалаберность", "небрежность", "халтура",
    "халатность"
]

SARCASM_INDICATORS = [
    "ну конечно", "конечно же", "ага", "прям", "ясно", "точно", "серьёзно", "да ладно", "вот это", "ну да",
    "как же", "прямо", "неужели", "верится", "охотно", "похоже", "да да", "верим", "безусловно",
    "естественно", "разумеется", "само собой", "ещё бы", "а то", "точно-точно", "вот прям", "ну-ну", "ну давай",
    "ох уж", "ну всё", "всё ясно", "всё понятно", "понятненько", "вот же",
    "ничего себе", "вот это да", "вот это поворот", "вот так вот", "ну и ну", "ну и дела",
    "вот тебе и", "вот тебе на", "ну ты даёшь", "да ты шутишь", "да ты серьёзно", "вот так сюрприз",
    "вот так новость", "вот так история", "вот так дела", "ну и как", "ну и что", "ну и зачем",
    "ну и почему", "какая щедрость"
]

# Добавляем больше эмодзи
POSITIVE_EMOJIS = [
    "😊", "😀", "😁", "😂", "🤣", "😍", "🥰", "😘", "😻", "💖", "💕", "💓", "💗", "💘", "🎉", "🥳", "👏", "👍", "🌟", "✨",
    "😄", "😆", "😇", "🤗", "💞", "💝", "💟", "🎈", "🎊", "🎁", "🌈", "☀️", "🌞", "🌺", "🌸", "🌼", "💐", "😽", "😺", "🥳",
    "👌", "✌️", "🙌", "💃", "🕺", "🎶", "🎵", "🥂", "🍾", "🍰", "🍭", "🍬", "💎", "⭐", "🌟", "💥", "🔥"
]

NEGATIVE_EMOJIS = [
    "😡", "😠", "😤", "😢", "😭", "😞", "😟", "😖", "😓", "💔", "👎", "😩", "😫", "🤬", "🤯", "💩", "🖕", "😵", "🤢",
    "😣", "😥", "😰", "😨", "😱", "😳", "🤮", "😿", "🙀", "💀", "☠️", "⚰️", "🪦", "🔪", "🩸", "😪", "😴", "😒", "😕",
    "😬", "😓", "🥀", "🌧️", "⛈️", "🌪️", "❄️", "🥶", "🔥", "💥", "💣", "🕳️", "🪓", "🤕", "🤒", "🤧", "🤥", "👿"
]

SARCASM_EMOJIS = [
    "🙄", "😏", "🤦", "🤷", "😒", "🤨", "😆", "🤓", "😜", "🤡",
    "😎", "😉", "😝", "😛", "🤪", "🤫", "🤭", "🤥", "🧐", "😈", "👻", "👽", "🤖", "🎭", "🃏", "🎰", "🎲", "🎯", "🔮"
]

# Эмодзи для смешанных эмоций
MIXED_EMOJIS = [
    "🥺", "😅", "😓", "😊😢", "😂😭", "😍💔", "😘😞", "💖😟", "🎉😥", "🥳😰", "👏😖", "👍😓", "🌟😢", "✨😭",
    "😄😢", "😆😞", "🤗💔", "💞😓", "💝😟", "🎈😥", "🎊😰", "🎁😖", "🌈😢", "☀️😭"
]

In [4]:

def clean_text(text):
    """Очистка текста от лишних символов и пробелов, но сохраняем эмодзи."""
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\w\s😀-🙏]', '', text)  # Удаляем всё, кроме слов, пробелов и эмодзи
    text = re.sub(r'\s+', ' ', text).strip()  # Удаляем лишние пробелы
    # Фильтруем слишком короткие комментарии (менее 3 слов)
    if len(text.split()) < 3:
        return ""
    return text

def has_sarcasm_indicators(text):
    """Проверяет наличие индикаторов сарказма/иронии."""
    text_lower = text.lower()
    for indicator in SARCASM_INDICATORS:
        if indicator in text_lower:
            return True
    return False

def auto_label(text):
    """Автоматическая разметка текста с учётом новых категорий, включая смешанные эмоции."""
    # Проверяем эмодзи
    for emoji in SARCASM_EMOJIS:
        if emoji in text:
            return "сарказм_ирония"

    for emoji in MIXED_EMOJIS:
        if emoji in text:
            return "смешанные_эмоции"

    for emoji in POSITIVE_EMOJIS:
        if emoji in text:
            return "позитив"

    for emoji in NEGATIVE_EMOJIS:
        if emoji in text:
            return "негатив"

    # Удаляем эмодзи для проверки текста
    text_no_emojis = re.sub(r'[😀-🙏]', '', text).lower()
    words = text_no_emojis.split()

    # Проверяем сарказм/иронию
    if has_sarcasm_indicators(text_no_emojis):
        for word in words:
            if word in POSITIVE_WORDS or word in NEGATIVE_WORDS:
                return "сарказм_ирония"

    # Проверяем ключевые слова
    has_positive = False
    has_negative = False
    for word in words:
        if word in POSITIVE_WORDS:
            has_positive = True
        if word in NEGATIVE_WORDS:
            has_negative = True

    # Определяем метку
    if has_positive and has_negative:
        return "смешанные_эмоции"  # Если есть и позитив, и негатив, но это не сарказм — смешанные эмоции
    elif has_positive:
        return "позитив"
    elif has_negative:
        return "негатив"
    return "нейтрально"

In [5]:

def collect_comments(group_screen_name, token, max_posts=100, max_comments_per_post=100):
    """Собирает комментарии из группы ВКонтакте."""
    try:
        # Инициализация VK API
        vk_session = vk_api.VkApi(token=token)
        vk = vk_session.get_api()

        # Проверяем информацию о группе
        group_info = vk.groups.getById(group_id=group_screen_name)
        group_id = group_info[0]["id"]
        print(f"Группа: {group_info[0]['name']}, ID: {group_id}, Закрытая: {group_info[0]['is_closed']}")
        if group_info[0]["is_closed"] != 0:
            print("Группа закрытая, доступ к данным ограничен.")
            return []
        if group_id > 0:
            group_id = -group_id

        # Получаем посты
        posts = vk.wall.get(owner_id=group_id, count=max_posts)["items"]
        print(f"Получено {len(posts)} постов.")

        data = []
        for post in posts:
            post_id = post["id"]
            print(f"Обработка поста {post_id}...")
            try:
                # Получаем комментарии к посту
                comments = vk.wall.getComments(
                    owner_id=group_id,
                    post_id=post_id,
                    count=max_comments_per_post,
                    extended=1,
                    fields="profiles"
                )
                time.sleep(0.5)  # Задержка 0.5 секунды между запросами
                if "items" not in comments:
                    print(f"Нет комментариев для поста {post_id}.")
                    continue

                for item in comments["items"]:
                    text = item.get("text", "")
                    if not text:
                        continue
                    cleaned_text = clean_text(text)
                    if not cleaned_text:
                        continue
                    label = auto_label(cleaned_text)
                    data.append({"text": cleaned_text, "label": label})
            except Exception as e:
                print(f"Ошибка при получении комментариев для поста {post_id}: {e}")
                continue

        return data

    except Exception as e:
        print(f"Ошибка при сборе данных: {e}")
        return []

def save_to_csv(data, filename="sentiment_dataset.csv"):
    """Сохраняет данные в CSV."""
    df = pd.DataFrame(data, columns=["text", "label"])
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"Датасет сохранён в {filename}. Всего записей: {len(df)}")

# Укажите токен VK и название группы
TOKEN = "219d7b3e219d7b3e219d7b3ed822b0844c2219d219d7b3e467bf727aa63bd40cfb3a0cb"
GROUP_SCREEN_NAME = "vk"

# Собираем данные
data = collect_comments(GROUP_SCREEN_NAME, TOKEN, max_posts=100, max_comments_per_post=100)

# Сохраняем в CSV
save_to_csv(data, "sentiment_dataset.csv")

Группа: ВКонтакте, ID: 22822305, Закрытая: 0
Получено 100 постов.
Обработка поста 1604841...
Обработка поста 1604782...
Обработка поста 1604440...
Обработка поста 1603812...
Обработка поста 1603321...
Обработка поста 1603073...
Обработка поста 1602230...
Обработка поста 1600899...
Обработка поста 1600187...
Обработка поста 1599555...
Обработка поста 1599077...
Обработка поста 1598343...
Обработка поста 1597292...
Обработка поста 1596766...
Обработка поста 1596463...
Обработка поста 1595281...
Обработка поста 1594978...
Обработка поста 1594111...
Обработка поста 1593717...
Обработка поста 1593596...
Обработка поста 1592635...
Обработка поста 1592462...
Обработка поста 1590899...
Обработка поста 1589801...
Обработка поста 1588812...
Обработка поста 1588043...
Обработка поста 1587425...
Обработка поста 1587105...
Обработка поста 1584867...
Обработка поста 1583409...
Обработка поста 1580085...
Обработка поста 1579266...
Обработка поста 1577468...
Обработка поста 1575041...
Обработка поста 

In [6]:

import pandas as pd

# Загружаем датасет
df = pd.read_csv("sentiment_dataset.csv")

# Проверяем распределение классов
print("\nИсходное распределение классов:")
print(df["label"].value_counts())
print("\nПроцентное распределение классов:")
print(df["label"].value_counts(normalize=True) * 100)

# Аугментация данных для редких классов (кроме нейтрально)
mixed_emotions_examples = pd.DataFrame({
    "text": [
        "Я рад, но немного грустно 😊😢",
        "Счастье с оттенком тоски, как странно 🌟😟",
        "Классный день, но всё равно что-то не так 🎉😞",
        "Люблю, но иногда так раздражает 💕😤",
        "Всё супер, но есть небольшое разочарование 😍💔",
        "Улыбаюсь, но внутри грусть 😂😭",
        "Весело, но в душе пустота 🎈😢",
        "Отличные новости, но всё равно тревожно 🌞😰",
        "Праздник, но настроение не очень 🎊😓",
        "Красиво, но немного грустно 🌸😟"
    ] * 10,  # Умножаем, чтобы получить 100 примеров
    "label": ["смешанные_эмоции"] * 100
})

# Объединяем с исходным датасетом
df = pd.concat([df, mixed_emotions_examples], ignore_index=True)

# Уменьшаем количество примеров для нейтрально (undersampling)
neutral_df = df[df["label"] == "нейтрально"].sample(n=500, random_state=42) if len(df[df["label"] == "нейтрально"]) > 500 else df[df["label"] == "нейтрально"]
other_df = df[df["label"] != "нейтрально"]
df_balanced = pd.concat([neutral_df, other_df], ignore_index=True)

# Сохраняем сбалансированный датасет
df_balanced.to_csv("sentiment_dataset_balanced.csv", index=False)

# Проверяем новое распределение
print("\nНовое распределение классов после аугментации и undersampling:")
print(df_balanced["label"].value_counts())
print("\nПроцентное распределение классов:")
print(df_balanced["label"].value_counts(normalize=True) * 100)


Исходное распределение классов:
label
нейтрально          4720
позитив              745
негатив              739
сарказм_ирония       278
смешанные_эмоции     104
Name: count, dtype: int64

Процентное распределение классов:
label
нейтрально          71.667173
позитив             11.311874
негатив             11.220771
сарказм_ирония       4.221075
смешанные_эмоции     1.579107
Name: proportion, dtype: float64

Новое распределение классов после аугментации и undersampling:
label
позитив             745
негатив             739
нейтрально          500
сарказм_ирония      278
смешанные_эмоции    204
Name: count, dtype: int64

Процентное распределение классов:
label
позитив             30.210868
негатив             29.967559
нейтрально          20.275750
сарказм_ирония      11.273317
смешанные_эмоции     8.272506
Name: proportion, dtype: float64


In [7]:

# Загружаем сбалансированный датасет
df_balanced = pd.read_csv("sentiment_dataset_balanced.csv")

# Сохраняем финальный датасет
df_balanced.to_csv("sentiment_dataset_final.csv", index=False)
print("\nФинальный датасет сохранён как 'sentiment_dataset_final.csv'")

# Проверяем финальное распределение
print("\nФинальное распределение классов:")
print(df_balanced["label"].value_counts())
print("\nПроцентное распределение классов:")
print(df_balanced["label"].value_counts(normalize=True) * 100)


Финальный датасет сохранён как 'sentiment_dataset_final.csv'

Финальное распределение классов:
label
позитив             745
негатив             739
нейтрально          500
сарказм_ирония      278
смешанные_эмоции    204
Name: count, dtype: int64

Процентное распределение классов:
label
позитив             30.210868
негатив             29.967559
нейтрально          20.275750
сарказм_ирония      11.273317
смешанные_эмоции     8.272506
Name: proportion, dtype: float64


In [8]:
# Проверка содержимого датасета
df_check = pd.read_csv("sentiment_dataset_final.csv")
print(f"Размер датасета: {len(df_check)} записей")
print("Первые 5 строк:")
print(df_check.head())

Размер датасета: 2466 записей
Первые 5 строк:
                                                text       label
0  пока нет ни одного удобного и разумного нововв...  нейтрально
1  задолбали эти каракули строчат и строчат То эт...  нейтрально
2                               Сова никогда не спит  нейтрально
3                        Не работает Ошибка загрузки  нейтрально
4  Я не понял Что это вы меня заблокировали до 1 ...  нейтрально


In [9]:
import transformers
import torch
print(f"Версия transformers: {transformers.__version__}")
print(f"Версия torch: {torch.__version__}")
print(f"Доступность GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Имя GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU (выделено/всего): {torch.cuda.memory_allocated(0)/1024**3:.2f} GB / {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")

# Очистка памяти GPU (на всякий случай)
torch.cuda.empty_cache()
gc.collect()
print("Память очищена.")

Версия transformers: 4.51.3
Версия torch: 2.6.0+cu124
Доступность GPU: True
Имя GPU: Tesla T4
Память GPU (выделено/всего): 0.00 GB / 14.74 GB
Память очищена.


In [10]:

import transformers
import torch
import numpy as np
import gc
from sklearn.metrics import f1_score  # Добавляем импорт f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

print(f"Версия transformers: {transformers.__version__}")
print(f"Версия torch: {torch.__version__}")
print(f"Доступность GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Имя GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU (выделено/всего): {torch.cuda.memory_allocated(0)/1024**3:.2f} GB / {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")

# Очистка памяти GPU
torch.cuda.empty_cache()
gc.collect()
print("Память очищена.")

# Определение классов и функций
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        print(f"Создаю датасет: {len(self.texts)} записей.")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def compute_class_weights(labels):
    weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
    return torch.tensor(weights, dtype=torch.float)

def load_data(file_path):
    print(f"Загружаю датасет из {file_path}...")
    df = pd.read_csv(file_path)
    print(f"Датасет загружен. Размер: {len(df)} записей.")

    if df['text'].isnull().any() or df['label'].isnull().any():
        print("В датасете есть пропущенные значения! Удаляю их...")
        df = df.dropna()
        print(f"После удаления пропущенных значений: {len(df)} записей.")

    texts = df["text"].values
    labels = df["label"].values
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    print(f"Количество уникальных классов: {len(label_encoder.classes_)}")
    return texts, labels, label_encoder

class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    f1 = f1_score(labels, predictions, average="weighted")  # Теперь f1_score определён
    return {
        "accuracy": accuracy,
        "f1": f1
    }

def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Используемое устройство: {device}")

    texts, labels, label_encoder = load_data("sentiment_dataset_final.csv")
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=42
    )
    print(f"Размер тренировочной выборки: {len(train_texts)}")
    print(f"Размер валидационной выборки: {len(val_texts)}")

    model_name = "DeepPavlov/rubert-base-cased"
    print(f"Загружаю токенизатор {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"Загружаю модель {model_name}...")
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(label_encoder.classes_),
        hidden_dropout_prob=0.3,
        attention_probs_dropout_prob=0.3
    )
    model.to(device)
    print(f"Модель загружена и перемещена на устройство {device}.")

    train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
    val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
    print("Датасеты для обучения и валидации созданы.")

    class_weights = compute_class_weights(labels)
    print(f"Веса классов: {class_weights}")

    training_args = TrainingArguments(
        output_dir="./sentiment_model_final",
        num_train_epochs=7,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.1,
        learning_rate=3e-5,
        logging_dir="./logs",
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=1,
        report_to="none",
    )
    print("Параметры обучения настроены.")

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        class_weights=class_weights
    )
    print("Тренер инициализирован.")

    print("Начинаю обучение...")
    try:
        trainer.train()
        print("Обучение завершено!")
    except Exception as e:
        print(f"Ошибка во время обучения: {e}")
        raise e

    model.save_pretrained("./sentiment_model_final")
    tokenizer.save_pretrained("./sentiment_model_final")
    print("Модель сохранена в './sentiment_model_final'")

    with open("label_encoder_final.pkl", "wb") as f:
        pickle.dump(label_encoder, f)
    print("Label encoder сохранён в 'label_encoder_final.pkl'")

    torch.cuda.empty_cache()
    gc.collect()

    return model, tokenizer, label_encoder

# Запускаем обучение
try:
    model, tokenizer, label_encoder = train_model()
    print("Функция train_model успешно выполнена!")
except Exception as e:
    print(f"Ошибка в train_model: {e}")

Версия transformers: 4.51.3
Версия torch: 2.6.0+cu124
Доступность GPU: True
Имя GPU: Tesla T4
Память GPU (выделено/всего): 0.00 GB / 14.74 GB
Память очищена.
Используемое устройство: cuda
Загружаю датасет из sentiment_dataset_final.csv...
Датасет загружен. Размер: 2466 записей.
Количество уникальных классов: 5
Размер тренировочной выборки: 1972
Размер валидационной выборки: 494
Загружаю токенизатор DeepPavlov/rubert-base-cased...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель загружена и перемещена на устройство cuda.
Создаю датасет: 1972 записей.
Создаю датасет: 494 записей.
Датасеты для обучения и валидации созданы.
Веса классов: tensor([0.6674, 0.9864, 0.6620, 1.7741, 2.4176])
Параметры обучения настроены.
Тренер инициализирован.
Начинаю обучение...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.624700,1.603691,0.307692,0.174741
2,1.534900,1.551378,0.289474,0.211363
3,1.241600,1.409136,0.378543,0.395637
4,1.121900,1.224616,0.469636,0.473062
5,0.957800,1.056502,0.564777,0.582265
6,0.759400,1.024088,0.597166,0.622268
7,0.681900,1.077479,0.609312,0.624332


Обучение завершено!
Модель сохранена в './sentiment_model_final'
Label encoder сохранён в 'label_encoder_final.pkl'
Функция train_model успешно выполнена!


In [11]:
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def predict_sentiment(text, model, tokenizer, label_encoder, device):
    """Предсказывает тональность текста."""
    print(f"Тестирую текст: {text}")
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    encoding = {key: value.to(device) for key, value in encoding.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
        predicted_class = logits.argmax(dim=-1).item()

    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    print(f"Предсказанная тональность: {predicted_label}")
    return predicted_label

# Проверяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство для тестирования: {device}")

# Загружаем модель, токенизатор и label_encoder
print("Загружаю сохранённую модель и токенизатор...")
model = AutoModelForSequenceClassification.from_pretrained("./sentiment_model_final")
tokenizer = AutoTokenizer.from_pretrained("./sentiment_model_final")
with open("label_encoder_final.pkl", "rb") as f:
    label_encoder = pickle.load(f)
print("Модель, токенизатор и label_encoder загружены.")

model.to(device)

# Тестовые примеры
test_texts = [
    "Все хорошо",
    "Ужасный день, всё идёт не так",
    "Ничего особенного, обычный день",
    "Я счастлив, но немного грустно 😊😢"
]

print("\nТестирование модели:")
for text in test_texts:
    sentiment = predict_sentiment(text, model, tokenizer, label_encoder, device)
    print(f"Текст: {text}\nТональность: {sentiment}\n")

Используемое устройство для тестирования: cuda
Загружаю сохранённую модель и токенизатор...
Модель, токенизатор и label_encoder загружены.

Тестирование модели:
Тестирую текст: Все хорошо
Предсказанная тональность: позитив
Текст: Все хорошо
Тональность: позитив

Тестирую текст: Ужасный день, всё идёт не так
Предсказанная тональность: смешанные_эмоции
Текст: Ужасный день, всё идёт не так
Тональность: смешанные_эмоции

Тестирую текст: Ничего особенного, обычный день
Предсказанная тональность: позитив
Текст: Ничего особенного, обычный день
Тональность: позитив

Тестирую текст: Я счастлив, но немного грустно 😊😢
Предсказанная тональность: смешанные_эмоции
Текст: Я счастлив, но немного грустно 😊😢
Тональность: смешанные_эмоции



In [12]:

import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from google.colab import files

# Загружаем модель и токенизатор
print("Загружаю сохранённую модель и токенизатор...")
model = AutoModelForSequenceClassification.from_pretrained("./sentiment_model_final")
tokenizer = AutoTokenizer.from_pretrained("./sentiment_model_final")
print("Модель и токенизатор загружены.")

# Создаём словарь для хранения модели и токенизатора
model_tokenizer_dict = {
    "model": model,
    "tokenizer": tokenizer
}

# Сохраняем модель и токенизатор в файл .pkl
model_pkl_path = "/content/model_and_tokenizer.pkl"
with open(model_pkl_path, "wb") as f:
    pickle.dump(model_tokenizer_dict, f)
print(f"Модель и токенизатор сохранены в {model_pkl_path}")

# Проверяем, что label_encoder уже сохранён
label_encoder_pkl_path = "/content/label_encoder_final.pkl"
import os
if os.path.exists(label_encoder_pkl_path):
    print(f"Label encoder уже сохранён в {label_encoder_pkl_path}")
else:
    raise FileNotFoundError("Файл label_encoder_final.pkl не найден! Убедись, что ячейка 8 была выполнена.")

# Скачиваем оба файла
files.download(model_pkl_path)
print("Файл model_and_tokenizer.pkl скачан.")
files.download(label_encoder_pkl_path)
print("Файл label_encoder_final.pkl скачан.")

Загружаю сохранённую модель и токенизатор...
Модель и токенизатор загружены.


KeyboardInterrupt: 